# 🔍 Building an AI-Powered Web Search Agent with OpenAI and Tavily 🚀

Hey there! Welcome to this exciting guide where we'll create something awesome - a smart search agent that combines the power of OpenAI's language models with Tavily's search capabilities! 🌟 

## 🎯 What We'll Build

We're going to create a super cool search agent that can:
1. 🌐 Search the web in real-time for accurate information
2. 🧠 Use OpenAI's powerful GPT models to understand and process search results
3. ⚡ Provide contextual and up-to-date responses to queries

## ✅ Prerequisites

Before we jump in, make sure you have these things ready:
- 🔑 An OpenAI API key
- 🎯 A Tavily API key (get one at tavily.com)

## 🎮 Part 1: Setting Up Our Environment

First things first - let's get our tools ready! We'll need to install the Tavily Python package to interact with their search API:


In [1]:
%pip install tavily-python

  Using cached tavily_python-0.5.1-py3-none-any.whl.metadata (91 kB)
  Using cached tiktoken-0.9.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
Using cached tavily_python-0.5.1-py3-none-any.whl (43 kB)
Using cached tiktoken-0.9.0-cp312-cp312-macosx_11_0_arm64.whl (1.0 MB)
Note: you may need to restart the kernel to use updated packages.



Now, let's set up our API keys (remember to keep these safe and never share them!):


In [2]:
TAVILY_API = "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"


## 🛠️ Part 2: Building Our Search Tools

Let's create the foundation of our search agent! We'll define a set of tools that our AI can use to search the web:

In [3]:
import json
from openai import OpenAI
from tavily import TavilyClient
import pprint

In [4]:
# Initialize Tavily
tavily = TavilyClient(api_key=TAVILY_API)

# Search query
query = "Get me flights from barcelona to madrid on the 3/3"
tavily.search(query, search_depth="basic")

{'query': 'Get me flights from barcelona to madrid on the 3/3',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'title': 'Direct (non-stop) flights from Barcelona to Madrid',
   'url': 'https://www.flightsfrom.com/BCN-MAD',
   'content': 'Air Europa flight schedule from Barcelona to Madrid. All weekly departures with Air Europa. The week calendar shows every flight departure from Barcelona (BCN) to Madrid (MAD). Click on a blue date to see a list of flights. W10 (Mar 3 - Mar 9) W08 (Feb 17 - Feb 23) W09 (Feb 24 - Mar 2)',
   'score': 0.76002824,
   'raw_content': None},
  {'title': '$18 Cheap Flights from Barcelona (BCN) to Madrid (MAD)',
   'url': 'https://www.expedia.com/lp/flights/bcn/mad/barcelona-to-madrid',
   'content': 'Cheap Flights from Barcelona (BCN) to Madrid (MAD) start at $22 for one-way and $38 for round trip. Earn your airline miles on top of our rewards!',
   'score': 0.6188962,
   'raw_content': None},
  {'title': '$23 CHEAP FLIGHTS from B

In [21]:
TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string"}
                },
                "required": ["query"]
            },
        },
    },
]

In [22]:
def search_web(query):
    # Initialize Tavily
    tavily = TavilyClient(api_key=TAVILY_API)
    return "\n".join(result["content"] for result in tavily.search(query, search_depth="basic")["results"])

In [23]:
search_web("What is the current weather in Tokyo")

"{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.6895, 'lon': 139.6917, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1739952313, 'localtime': '2025-02-19 17:05'}, 'current': {'last_updated_epoch': 1739952000, 'last_updated': '2025-02-19 17:00', 'temp_c': 7.2, 'temp_f': 45.0, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 22.4, 'wind_kph': 36.0, 'wind_degree': 345, 'wind_dir': 'NNW', 'pressure_mb': 1012.0, 'pressure_in': 29.88, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 16, 'cloud': 0, 'feelslike_c': 2.5, 'feelslike_f': 36.5, 'windchill_c': 0.7, 'windchill_f': 33.3, 'heatindex_c': 5.9, 'heatindex_f': 42.5, 'dewpoint_c': -14.3, 'dewpoint_f': 6.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 27.2, 'gust_kph': 43.8}}\nWeather in Tokyo in February 2025 (Tōkyō-to) - Detailed Weather Forecast for a Month Add the current city Search Weather World Tōkyō-to Weather in T


## 🎓 Part 3: Creating Our AI Agent

Now comes the exciting part! Let's create our AI agent that can understand questions and use our search tools to find answers:


In [24]:
from datetime import datetime
messages = [
    {"role": "system", "content": f"You are a helpful trip advisor assistant that can search the web to create a plan for a trip. A basic trip requires: 1. Transportation 2. Accommodation 3. Food 4. Activities. Make as many tool calls as needed to create a comprehensive plan. Current date: {datetime.now().strftime('%Y-%m-%d')}"},
    {"role": "user", "content": "Plan a trip to Barcelona for 3 days"}
]

In [25]:
def invoke_model(messages):
    # Initialize the OpenAI client
    client = OpenAI()

    # Make a ChatGPT API call with tool calling
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages
    )

    return completion.choices[0].message.content

In [28]:
# Initialize the OpenAI client
client = OpenAI()

# Make a ChatGPT API call with tool calling
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    tools=TOOLS,
    messages=messages
)

response = completion.choices[0].message
pprint.pprint(response.tool_calls)

# Parse the response to get the tool call arguments
if response.tool_calls:
    # Process each tool call
    for tool_call in response.tool_calls:
        # Get the tool call arguments
        tool_call_arguments = json.loads(tool_call.function.arguments)
        if tool_call.function.name == "web_search":
            print("Searching for", tool_call_arguments)
            search_results = search_web(tool_call_arguments["query"])
            messages.append({"role": "assistant", "content": f"{tool_call_arguments["query"]}: {search_results}"})
    print(invoke_model(messages))

else:
    # If there are no tool calls, return the response content
    print(response.content)

[ChatCompletionMessageToolCall(id='call_bJbyXHJBHkyZMoquJDbnpg90', function=Function(arguments='{"query": "best transportation options to Barcelona 2025"}', name='web_search'), type='function'),
 ChatCompletionMessageToolCall(id='call_cxywLLS3Hc1liWGo3aRpAjrx', function=Function(arguments='{"query": "best hotels in Barcelona February 2025"}', name='web_search'), type='function'),
 ChatCompletionMessageToolCall(id='call_bN0m2OZqMRYture2lXMDM5cU', function=Function(arguments='{"query": "best restaurants in Barcelona February 2025"}', name='web_search'), type='function'),
 ChatCompletionMessageToolCall(id='call_uJi2YpSHgKmtKcBdGyepNhpP', function=Function(arguments='{"query": "top activities to do in Barcelona February 2025"}', name='web_search'), type='function')]
Searching for {'query': 'best transportation options to Barcelona 2025'}
Searching for {'query': 'best hotels in Barcelona February 2025'}
Searching for {'query': 'best restaurants in Barcelona February 2025'}
Searching for {'q

In [27]:
messages

[{'role': 'system',
  'content': 'You are a helpful trip advisor assistant that can search the web to create a plan for a trip. A basic trip requires: 1. Transportation 2. Accommodation 3. Food 4. Activities. Make as many tool calls as needed to create a comprehensive plan. Current date: 2025-02-19'},
 {'role': 'user', 'content': 'Plan a trip to Barcelona for 3 days'},
 {'role': 'assistant',
  'content': "transportation options to Barcelona: In this article, we will discuss getting around Barcelona, including metro, bus, taxi, and many more transportation options. If you’re seeking an efficient and pocket-friendly way to traverse the city, you can’t overlook public transportation in Barcelona. For those visiting the city and eager to discover its many treasures, using public transportation in Barcelona is likely the optimal choice. Another jewel in the crown of transportation in Barcelona is the city bus. Barcelona is a bike-friendly city, and cycling around is the perfect mode of tran